Latest update: 2022-12-09 16:39:50.031469

<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/02_Upload_new_footage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorials #2: Upload new footage</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>

# Set up and requirements

## Install kso_data_management and its requirements

In [1]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    !pip install -q --upgrade pip
    !pip install -q -r kso-data-management/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso_dev/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys

    # Install requirements
    !pip install -q --no-warn-script-location --upgrade pip
    !pip install -qr ../requirements.txt

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

Running locally... you're good to go!


### Import Python packages

In [2]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
# os.chdir("kso_dev")
# sys.path.append("/usr/src/app/kso_dev")
sys.path.insert(1, "/usr/src/app/kso_dev")


# Enable third party widgets
if IN_COLAB:
    from google.colab import output

    output.enable_custom_widget_manager()

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t2_utils as t2
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as serv_utils

# import kso_utils.server_utils as serv_utils

print("Packages loaded successfully")

Packages loaded successfully


## Choose your project

In [3]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

## Initiate sql and get server or local storage details

In [4]:
# Save project name
project = p_utils.find_project(project_name=project_name.value)
# Initiate db
db_info_dict = t_utils.initiate_db(project)

INFO:root:Koster_Seafloor_Obs loaded succesfully


Enter your username for SNIC server········
Enter your password for SNIC server········


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:root:Updated sites
INFO:root:Updated movies
INFO:root:Updated species


# Select the survey linked to the videos

In [ ]:
survey_i = t2.select_survey(db_info_dict)

In [ ]:
survey_name = t2.confirm_survey(survey_i, db_info_dict)

## Select new deployments

To save time you can select multiple deployments **recorded on the same day**

In [ ]:
deployment_selected, survey_row, survey_server_name = t2.select_deployment(
    project=project, db_info_dict=db_info_dict, survey_i=survey_i
)

## Specify the date of the deployments

In [ ]:
deployment_date = t2.select_eventdate(survey_row)

## Check the database to avoid deployment duplicates

In [ ]:
deployment_names = t2.check_deployment(
    deployment_selected, deployment_date, survey_server_name, db_info_dict, survey_i
)

## Update new deployment files

In [ ]:
movie_files_server = t2.update_new_deployments(
    deployment_filenames=deployment_names,
    db_info_dict=db_info_dict,
    survey_server_name=survey_server_name,
    deployment_date=deployment_date,
)

## Specify deployment details

In [ ]:
deployment_info = t2.record_deployment_info(deployment_names, db_info_dict)

## Review deployment  details

In [ ]:
new_deployment_row = t2.confirm_deployment_details(
    deployment_names,
    survey_server_name,
    db_info_dict,
    survey_i,
    deployment_info,
    movie_files_server,
    deployment_date,
)

# !!!Only pass this point if deployment details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
t2.upload_concat_movie(db_info_dict, new_deployment_row)

# Upload movies to SNIC

### Choose files to upload

In [5]:
movie_list = t2.choose_new_videos_to_upload()

Choose the file that you want to upload: 


FileChooser(path='/usr/src/app/kso_dev/tutorials', filename='', title='HTML(value='', layout=Layout(display='n…

Showing paths to the selected movies:
Rerun cell to reset
--------------


Button(description='Add selected file', style=ButtonStyle())

Output()

### Upload the videos and add metadata

In [6]:
new_movie_rows_sheet = t2.upload_new_movies_to_snic(db_info_dict, movie_list)

movie to be uploaded: movie_1_processed.mp4


100%|##########| 30.8k/30.8k [00:00<00:00, 1.88Mb/s]

movie uploaded

All movies uploaded:

Complete this sheet by filling the missing info on the movie you just uploaded


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='127'), Cell(column…

### concatenate previous data with new rows


In [7]:
df_with_new_rows = t2.add_new_rows_to_csv(db_info_dict, new_movie_rows_sheet)

### Create the local updated csv

In [12]:
# update the csv

updated_csv = "movies_koster_updated.csv"
updated_csv_path = (
    "/cephyr/NOBACKUP/groups/snic2021-6-9/tmp_dir/local_dir_dev/" + updated_csv
)
orig_csv = db_info_dict["local_movies_csv"].name
csv_with_new_rows = df_with_new_rows.to_csv(updated_csv_path)

/cephyr/NOBACKUP/groups/snic2021-6-9/tmp_dir/local_dir_devmovies_koster_updated.csv


100%|##########| 10.2k/10.2k [00:00<00:00, 1.62Mb/s]


### upload the csv to the server

In [ ]:
serv_utils.update_csv_server(project, db_info_dict, orig_csv, updated_csv)

In [ ]:
# END

### Todo:
multiselect videos